<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Data_Cleaning2(FJC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Causal effect of Patent Litigation Outcomes on Firm Financials


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')

<ipython-input-2-caae5146eea8>:1: DtypeWarning: Columns (1,2,9,10,11,13,19,20,23,24,26,38,39,40,41,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  JFC_1988_Present = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/Civil_1988_to_Present.txt', sep='\t', encoding='latin1')


In [3]:
#ISOLATING PANTENT AND TRADENARK INFRINGEMENT SUITES, AND ONLY ORIGINAL PROCEEDINGS
#Patents_1988_forwards = pd.DataFrame(JFC_1988_Present[JFC_1988_Present['NOS'].isin([830, 840])]) #only patent/trademark cases
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ORIGIN'] == 1] #only original filings
#Patents_1988_forwards  = Patents_1988_forwards[Patents_1988_forwards ['ARBIT'].astype(str) == '-8'] #no formal arbitration courts

clean_JFC_patents = JFC_1988_Present.copy()

#removing columns that aren't relavent or mostly empty
clean_JFC_patents.drop(columns=['ORIGIN','TITL', 'SECTION', 'SUBSECT', 'ARBIT',
                                'FILEJUDG', 'FILEMAG', 'MDLDOCK',  'TRANSDAT',
                                'TRANSOFF', 'TRANSDOC', 'TRANSORG', 'TDATEUSE',
                                'TRCLACT', 'TERMJUDG', 'TERMMAG', 'DJOINED',
                                'PRETRIAL', 'TRIBEGAN', 'TRIALEND', 'AMTREC',
                                'TRMARB', 'RESIDENC', 'PROSE', 'IFP'],
                                 inplace=True)

In [4]:
clean_JFC_patents.head()

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,NOS,JURY,CLASSACT,...,COUNTY,PLT,DEF,TERMDATE,PROCPROG,DISP,NOJ,JUDGMENT,STATUSCD,TAPEYEAR
0,0,90,1,7101462,07/21/1971,07/01/1971,3,440,-8,1,...,0,CAMPBELL,RODGERS,11/17/1985,6,9,3,1,NaN,1988
1,0,90,1,7300974,05/17/1973,02/01/1983,2,890,-8,-8,...,0,MCKENZIE ETAL,MCCORMICK ETAL,04/18/1988,4,17,1,1,NaN,1988
2,0,90,1,7601326,01/31/1985,06/01/1985,2,440,-8,-8,...,0,HOBSON ETAL,WILSON ETAL,03/21/1988,10,13,-8,-8,NaN,1988
3,0,90,1,7700081,09/19/1984,05/01/1986,2,442,-8,-8,...,0,SEGAR,LEVI,10/19/1985,10,13,-8,-8,NaN,1988
4,0,90,1,7901559,06/15/1979,06/01/1979,2,550,-8,-8,...,0,PITTS,BELL,12/10/1987,12,6,0,2,NaN,1988


In [5]:
dummies = pd.get_dummies(clean_JFC_patents['NOS'])
clean_JFC_patents = pd.concat([clean_JFC_patents, dummies], axis=1)

dummies = pd.get_dummies(clean_JFC_patents['JUDGMENT'])
clean_JFC_patents = pd.concat([clean_JFC_patents, dummies], axis=1)

In [6]:
clean_JFC_patents = clean_JFC_patents.rename(columns={830: 'Patent', 840: 'Trademark'}).drop(columns='NOS')
clean_JFC_patents = clean_JFC_patents.rename(columns={-8: 'Missing',0: 'Missing', 1: 'Plaintiff', 2: 'Defendent',3: 'both', 4: 'Unkown' }).drop(columns='JUDGMENT')

In [7]:
clean_JFC_patents.loc[clean_JFC_patents['CLASSACT'] == -8, 'CLASSACT'] = 0

In [8]:
cases = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/cases.csv')
cases = cases[['case_row_id', 'case_number', 'case_name', 'case_cause', 'date_filed', 'date_closed', 'jurisdictional_basis']]

In [9]:
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['DOCKET'].astype(str).str[:2] + "-cv-" + clean_JFC_patents['DOCKET'].astype(str).str[2:]
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['JURIS'].astype(str) + "-" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[0] + "/" + clean_JFC_patents['FILEDATE'].astype(str).str.split("/").str[1] + "/" + clean_JFC_patents['unique_identifier'].astype(str)
clean_JFC_patents['unique_identifier'] = clean_JFC_patents['unique_identifier'] + "-" + clean_JFC_patents['TERMDATE'].astype(str)

#creating common unique identifier to match JFC dataset with
#creating common unique identifier to match JFC dataset with
cases['unique_identifier'] = cases['case_number'].astype(str).str.split(":", n=1).str[-1]
cases['unique_identifier'] = cases['date_filed'].astype(str).str.split("-").str[1] + "/" + cases['date_filed'].astype(str).str.split("-").str[2] + "/" +cases['unique_identifier'].astype(str)

for index, row in cases.iterrows():
    if row['jurisdictional_basis'] == "Federal Question":
        cases.loc[index, 'unique_identifier'] = "3-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Plaintiff":
        cases.loc[index, 'unique_identifier'] = "1-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "U.S. Government Defendant":
        cases.loc[index, 'unique_identifier'] = "2-" + row['unique_identifier']
    elif row['jurisdictional_basis'] == "Diversity":
        cases.loc[index, 'unique_identifier'] = "4-" + row['unique_identifier']

cases['unique_identifier'] = cases['unique_identifier'] + "-" + cases['date_closed'].astype(str).str.split("-").str[1] + "/" + cases['date_closed'].astype(str).str.split("-").str[2] + "/" + cases['date_closed'].astype(str).str.split("-").str[0]

In [10]:
clean_JFC_patents

,CIRCUIT,DISTRICT,OFFICE,DOCKET,FILEDATE,FDATEUSE,JURIS,JURY,CLASSACT,DEMANDED,...,950,990,992,Missing,Missing,Plaintiff,Defendent,both,Unkown,unique_identifier
0,0,90,1,7101462,07/21/1971,07/01/1971,3,-8,1,-8,...,False,False,False,False,False,True,False,False,False,3-07/21/71-cv-01462-11/17/1985
1,0,90,1,7300974,05/17/1973,02/01/1983,2,-8,0,-8,...,False,False,False,False,False,True,False,False,False,2-05/17/73-cv-00974-04/18/1988
2,0,90,1,7601326,01/31/1985,06/01/1985,2,-8,0,-8,...,False,False,False,True,False,False,False,False,False,2-01/31/76-cv-01326-03/21/1988
3,0,90,1,7700081,09/19/1984,05/01/1986,2,-8,0,-8,...,False,False,False,True,False,False,False,False,False,2-09/19/77-cv-00081-10/19/1985
4,0,90,1,7901559,06/15/1979,06/01/1979,2,-8,0,-8,...,False,False,False,False,False,False,True,False,False,2-06/15/79-cv-01559-12/10/1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10445834,11,3J,6,2400076,12/17/2024,12/01/2024,2,N,0,0,...,False,False,False,True,False,False,False,False,False,2-12/17/24-cv-00076-01/01/1900
10445835,11,3J,6,2400077,12/18/2024,12/01/2024,2,N,0,0,...,False,False,False,True,False,False,False,False,False,2-12/18/24-cv-00077-01/01/1900
10445836,11,3J,6,2400078,12/20/2024,12/01/2024,3,N,0,0,...,False,False,False,False,True,False,False,False,False,3-12/20/24-cv-00078-12/20/2024
10445837,11,3J,6,2400079,12/20/2024,12/01/2024,3,B,0,0,...,False,False,False,True,False,False,False,False,False,3-12/20/24-cv-00079-01/01/1900


In [11]:
cases.head()

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier
0,78028.0,4:92-cv-00055,"Melba Creek Mining, Inc. et al v. Sykes et al",No cause code entered,1992-12-22,1993-02-16,Federal Question,3-12/22/92-cv-00055-02/16/1993
1,75332.0,2:10-cv-00601,Bowhall v. Johnson & Johnson Inc. et al (MAG+),15:1126 Patent Infringement,2010-07-14,2010-10-12,Federal Question,3-07/14/10-cv-00601-10/12/2010
2,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998
3,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999
4,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018


In [12]:
merged = pd.merge(cases, clean_JFC_patents, on='unique_identifier', how='inner')

In [13]:
merged

,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,unique_identifier,CIRCUIT,DISTRICT,...,930,950,990,992,Missing,Missing,Plaintiff,Defendent,both,Unkown
0,75332.0,2:10-cv-00601,Bowhall v. Johnson & Johnson Inc. et al (MAG+),15:1126 Patent Infringement,2010-07-14,2010-10-12,Federal Question,3-07/14/10-cv-00601-10/12/2010,11,27,...,False,False,False,False,True,False,False,False,False,False
1,6908.0,1:98-cv-00596,Mettke v. Touchnet Information,35:183 Patent Infringement,1998-03-16,1998-09-21,Federal Question,3-03/16/98-cv-00596-09/21/1998,11,26,...,False,False,False,False,True,False,False,False,False,False
2,6933.0,1:98-cv-00654,"Monsanto Co. Inc. v. Styron, et al",28:1338 Patent Infringement,1998-06-29,1999-03-15,Federal Question,3-06/29/98-cv-00654-03/15/1999,11,28,...,False,False,False,False,True,False,False,False,False,False
3,86730.0,1:18-cv-00012,Sundesa LLC v. Eurark LLC,35:271 Patent Infringement,2018-02-13,2018-07-06,Federal Question,3-02/13/18-cv-00012-07/06/2018,8,60,...,False,False,False,False,False,True,False,False,False,False
4,30787.0,2:99-cv-00674,"Star Video Duplictns v. US Magnetics Corp, et al",15:1125 Trademark Infringement (Lanham Act),1999-04-09,1999-12-16,Federal Question,3-04/09/99-cv-00674-12/16/1999,9,70,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63537,28883.0,2:95-cv-00217,Wyoming Woolens v. Acorn Products Co,28:1338 Patent Infringement,1995-09-29,1995-10-06,Federal Question,3-09/29/95-cv-00217-10/06/1995,10,89,...,False,False,False,False,True,False,False,False,False,False
63538,101854.0,2:19-cv-00119,CUCU Inc v. Slickwraps Inc,35:145 Patent Infringement,2019-06-12,2019-10-10,Federal Question,3-06/12/19-cv-00119-10/10/2019,10,89,...,False,False,False,False,False,True,False,False,False,False
63539,29713.0,2:97-cv-00019,Chey Land & Cattle v. Crown Cork & Seal Co,35:271 Patent Infringement,1997-01-24,1997-07-25,Federal Question,3-01/24/97-cv-00019-07/25/1997,10,89,...,False,False,False,False,True,False,False,False,False,False
63540,39890.0,2:11-cv-00352,Weir Minerals Australia Ltd v. Megatrol Inc et al,35:271 Patent Infringement,2011-11-10,2012-11-15,Federal Question,3-11/10/11-cv-00352-11/15/2012,10,89,...,False,False,False,False,False,True,False,False,False,False


In [15]:
import wrds
db = wrds.Connection(wrds_username="sai047")

Loading library list...
Done


In [16]:
query = """
SELECT gvkey, conm
FROM comp.company
"""
comp_names = db.raw_sql(query)

In [17]:
comp_names

,gvkey,conm
0,001000,A & E PLASTIK PAK INC
1,001001,A & M FOOD SERVICES INC
2,001002,AAI CORP
3,001003,A.A. IMPORTING CO INC
4,001004,AAR CORP
...,...,...
55007,356128,JOINT STOCK COMPANY KASPI KZ
55008,356289,SOLARBANK CORP
55009,356687,ATHA ENERGY CORP
55010,356859,ACUREN CORP


In [18]:
gvkey_plt = pd.DataFrame(merged[['PLT']])
gvkey_def = pd.DataFrame(merged[['DEF']])

In [19]:
def clean_names(name):
    name = name.replace(",", '')  # Remove Commas
    name = name.replace(".", "")  # Remove periods
    name = name.replace("ET AL", "").strip()  # Remove 'ET AL' (case insensitive if already upper)
    return name

gvkey_def = pd.DataFrame(gvkey_def['DEF'].apply(clean_names)) # Apply to the 'DEF' column
gvkey_plt = pd.DataFrame(gvkey_plt['PLT'].apply(clean_names)) # Apply to the 'PLT' column
comp_names = pd.DataFrame(comp_names['conm'].apply(clean_names)) # Apply to the 'conm' column
merged['PLT'] = merged['PLT'].apply(clean_names)
merged['DEF'] = merged['DEF'].apply(clean_names)

In [20]:
comp_names['name'] = comp_names['conm']
gvkey_plt['name'] = gvkey_plt['PLT']
gvkey_def['name'] = gvkey_def['DEF']

In [21]:
gvkey_def

,DEF,name
0,JOHNSON & JOHNSON INC,JOHNSON & JOHNSON INC
1,TOUCHNET INFORMATION,TOUCHNET INFORMATION
2,STYRON,STYRON
3,EURARK LLC,EURARK LLC
4,US MAGNETICS CORP,US MAGNETICS CORP
...,...,...
63537,ACORN PRODUCTS CO,ACORN PRODUCTS CO
63538,SLICKWRAPS INC,SLICKWRAPS INC
63539,CROWN CORK & SEAL CO,CROWN CORK & SEAL CO
63540,MEGATROL INC,MEGATROL INC


In [23]:
temp = gvkey_def.merge(comp_names, on='name', how='inner')
temp = temp.merge(merged, on='DEF', how='inner')

In [27]:
temp

,DEF,name,conm,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,...,930,950,990,992,Missing,Missing,Plaintiff,Defendent,both,Unkown
0,SCYNEXIS INC,SCYNEXIS INC,SCYNEXIS INC,22604.0,1:13-cv-01566,Tet Systems GMBH & Co. KG v. Scynexis Inc.,35:271 Patent Infringement,2013-09-17,2013-10-15,Federal Question,...,False,False,False,False,False,True,False,False,False,False
1,INTERMEDICS INC,INTERMEDICS INC,INTERMEDICS INC,59316.0,4:93-cv-00626,"Medtronic, Inc., et al v. Intermedics, Inc., e...",35:145 Patent Infringement,1993-06-25,1996-10-08,Federal Question,...,False,False,False,False,True,False,False,False,False,False
2,INTERMEDICS INC,INTERMEDICS INC,INTERMEDICS INC,63761.0,5:93-cv-20544,"Ventritex, Inc., et al v. Intermedics, Inc.",35:145 Patent Infringement,1993-07-23,1996-09-04,Federal Question,...,False,False,False,False,True,False,False,False,False,False
3,INTERMEDICS INC,INTERMEDICS INC,INTERMEDICS INC,63762.0,5:93-cv-20566,"Ventritex, Inc., et al v. Intermedics, Inc., e...",35:145 Patent Infringement,1993-07-30,1997-07-11,Federal Question,...,False,False,False,False,True,False,False,False,False,False
4,INTERMEDICS INC,INTERMEDICS INC,INTERMEDICS INC,63514.0,5:90-cv-20481,"Ventritex, Inc. v. Intermedics, Inc.",35:271 Patent Infringement,1990-08-13,1991-12-23,Federal Question,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343171,NORDSTROM INC,NORDSTROM INC,NORDSTROM INC,98509.0,6:19-cv-00188,"Browse3D LLC v. Nordstrom, Inc.",35:271 Patent Infringement,2019-03-05,2019-09-05,Federal Question,...,False,False,False,False,False,True,False,False,False,False
343172,NORDSTROM INC,NORDSTROM INC,NORDSTROM INC,17714.0,1:11-cv-00203,"Cherdak v. Nordstrom, Inc.",35:281 Patent Infringement,2011-02-28,2011-08-03,Federal Question,...,False,False,False,False,False,True,False,False,False,False
343173,NORDSTROM INC,NORDSTROM INC,NORDSTROM INC,16964.0,1:10-cv-01029,"Cherdak v. Nordstrom, Inc.",35:1 Patent Non-Infringement - Declaratory Jud...,2010-09-15,2010-11-16,Federal Question,...,False,False,False,False,True,False,False,False,False,False
343174,NORDSTROM INC,NORDSTROM INC,NORDSTROM INC,16738.0,1:10-cv-00528,"Cherdak v. Nordstrom, Inc.",35:271 Patent Infringement,2010-05-21,2010-08-12,Federal Question,...,False,False,False,False,True,False,False,False,False,False


In [24]:
temp['case_number'].nunique()

7124

In [25]:
temp2 = gvkey_plt.merge(comp_names, on='name', how='inner')
temp2 = temp2.merge(merged, on='PLT', how='inner')

In [26]:
temp2

,PLT,name,conm,case_row_id,case_number,case_name,case_cause,date_filed,date_closed,jurisdictional_basis,...,930,950,990,992,Missing,Missing,Plaintiff,Defendent,both,Unkown
0,KIDDE INC,KIDDE INC,KIDDE INC,58881.0,4:85-cv-00399,"Kidde, Inc v. Fike Metal Products",28:1331 Fed. Question,1985-04-25,1989-10-23,Federal Question,...,False,False,False,False,False,False,False,True,False,False
1,DENTSPLY SIRONA INC,DENTSPLY SIRONA INC,DENTSPLY SIRONA INC,80014.0,1:17-cv-01041,"Dentsply Sirona Inc. et al v. Edge Endo, LLC",15:1126 Patent Infringement,2017-10-16,2021-06-03,Federal Question,...,False,False,False,False,False,False,True,False,False,False
2,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,59800.0,4:96-cv-02682,Sonoco Products Co v. Eagle Manufacturing,35:271 Patent Infringement,1996-09-03,1999-02-23,Federal Question,...,False,False,False,False,True,False,False,False,False,False
3,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,28342.0,2:93-cv-04929,"Sonoco Products Co v. Durabag Co Inc, et al",15:1126 Patent Infringement,1993-08-17,1996-08-30,Federal Question,...,False,False,False,False,True,False,False,False,False,False
4,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,SONOCO PRODUCTS CO,48309.0,3:90-cv-00627,"Sonoco Products Co. v. Vanguard Plastics, et al",28:1338 Patent Infringement,1990-07-18,1993-07-08,Federal Question,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91334,GILEAD SCIENCES INC,GILEAD SCIENCES INC,GILEAD SCIENCES INC,80060.0,1:17-cv-00036,"Gilead Sciences, Inc. v. Mylan Pharmaceuticals...",35:271 Patent Infringement,2017-03-03,2018-07-23,Federal Question,...,False,False,False,False,False,True,False,False,False,False
91335,GILEAD SCIENCES INC,GILEAD SCIENCES INC,GILEAD SCIENCES INC,84422.0,1:16-cv-00053,"Gilead Sciences, Inc. v. Mylan Pharmaceuticals...",35:271 Patent Infringement,2016-03-30,2018-07-23,Federal Question,...,False,False,False,False,False,True,False,False,False,False
91336,GILEAD SCIENCES INC,GILEAD SCIENCES INC,GILEAD SCIENCES INC,82560.0,1:17-cv-00217,"Gilead Sciences, Inc. v. Mylan Pharmaceuticals...",35:271 Patent Infringement,2017-12-15,2018-07-23,Federal Question,...,False,False,False,False,False,True,False,False,False,False
91337,CIMA LABS INC,CIMA LABS INC,CIMA LABS INC,16785.0,1:10-cv-00625,CIMA Labs Inc. et al v. Mylan Pharmaceuticals ...,35:271 Patent Infringement,2010-07-23,2016-03-09,Federal Question,...,False,False,False,False,False,True,False,False,False,False


In [28]:
temp2['case_number'].nunique()

3215

In [30]:
set_plt = set(temp2['case_number'].unique())
set_def = set(temp['case_number'].unique())

common = set_plt.intersection(set_def)
print(len(common))

380


In [31]:
3215 - 380 + 7124

9959